# Generalized Network Analysis Tutorial - Step 1

In this tutorial, we will use the Dynamic Network Analysis python package to explore the interactions between the OMP decarboxylase enzyme and its substrate, identifying clusters of amino acid residues that form domains, and active site residues that are important for binding and enzymatic activity.

The tutorial is divided into two jupyter notebooks. In this notebook, **Step 1**, we will analyze the MD trajectory for the OMP decarboxylase system and generate the network data used for analysis. 

![](./toolkit/figures/OMP-System-Network.png)
__Figure 1:__ The OMP decarboxylase enzyme and its substrate are shown on the left, where a cartoon representation was used to trace the backbone of the protein and a ball-and-stick representaion was used for the OMP. On the right, the network representation was overlayed on the system. The red tubes represent edges connecting network nodes with correlated motion, and the protein colors represent communities of nodes that show concerted movement. Red spheres represent structural water molecules automatically detected by the method.

In the accompanying notebook, **Step 2**, we will load the data generated in the Step 1, and create analysis plots and visualizations of the network. 

In **Step 3**, which is outside of the jupyter notebook environment, we will produce high-quality renderings of the network using the popular visualization software VMD.




**Important 1:**

If you are not familiar with Jupyter Notebooks, please take some time to watch a video tutorial, such as this:

https://www.youtube.com/watch?v=HW29067qVWk

If you are not familiar with VMD, you can find tutorials on how to use VMD here:

https://www.ks.uiuc.edu/Training/Tutorials/



**Important 2:**

Here, we use a language that is common among computational structural biophysicists. For instance:

The word **residue** is typically used to represent both an amino acid residue, or any other small molecule in the system, such as a ligand.

The word **segment** and also **segID** is typically used to represent a polymer chain, such as a protein, or a group of molecules in a lipid bilayer. Protein complexes would therefore be formed by multiple segments.

# Download Trajectory Data
The trajectory files used in this tutorial have approximately 500MB in size, and must be downloaded from the following link:

[Link to trajectory data](https://compbiophysics.auburn.edu/DyNetAn_Tutorial/data/DynamicNetworkAnalysis_MDdata.tar.gz) 

### Extract into the tutorial folder
After extracting the data, place the files in the *TutorialData* folder.


In [1]:
# Load the python package
import dynetan as dna

# Create the object that processes MD trajectories.
dnap = dna.proctraj.DNAproc()


# System information:
To get started, we need some basic information about the system:
- Work directory where we will find relevant files.
- Names of PSF and DCD files.
- segid's for the regions that will be used for network analysis.
- Name of the solvent residue(s) [default: TIP3]
- Number of windows into which the trajectory will be split. [default: 1]
- Name of atom(s) that will represent node(s) in each residue.
- Names of atoms in node groups.

**You can find a complete list of supported trajectory formats [here](https://www.mdanalysis.org/docs/documentation_pages/coordinates/init.html#supported-coordinate-formats):**


In [2]:
import os

# Path where input files are located and results will be written.
workDir = "./TutorialData/"

# PSF file name
psfFile = os.path.join(workDir, "decarboxylase.0.psf")

# DCD file name
dcdFiles = [os.path.join(workDir, "decarboxylase.1.dcd")]
# dcdFiles = [os.path.join(workDir, "decarboxylase.1.short.dcd")]

ligandSegID = "OMP"

# Segment IDs for regions that will be studied.
segIDs = ["OMP","ENZY"]

# Residue name for solvent molecule(s)
h2oName = ["TIP3"]

# Number of windows created from full simulation.
numWinds = 4

# Sampled frames per window
numSampledFrames = 10

#################################
### Extra configuration - Default values are given below

# Cutoff for contact map (In Angstroms)
cutoffDist = 4.5

# Minimum contact persistence (In ratio of total trajectory frames)
contactPersistence = 0.75

## Node Groups

In Network Analysis, each residue is represented by one or more "nodes", serving as proxies for groups of atoms which form the original residue (Figure 2). This approach lowers computational costs and noise. 

![](./toolkit/figures/nodeAtomGroup.png)
__Figure 2:__ (a) and (b) represent a chain of amino acids and a nucleotide, respectively. (c) and (d) represent the same molecules but highlight the difference between a node (alpha carbon in amino acids, and nitrogen and phosphate in nucleotides) and the group of atoms that the node represents.

The Dynamical Network Analysis package will automatically create one node per protein residue using the alpha carbon as the representative atom for the node (see Figure 2 a and c above). This selection can be modified for one or all amino acids by manually defining node groups.

For non-proteic residues such as nucleotides (see Figure 2 b and d above), the user must specify atom(s) which will represent at least one node per residue. For each node, we also need to know which heavy atoms from the residue compose each node group.

In [3]:

usrNodeGroups = {}

usrNodeGroups["TIP3"] = {}
usrNodeGroups["TIP3"]["OH2"] = set("OH2 H1 H2".split())

usrNodeGroups["OMP"] = {}
usrNodeGroups["OMP"]["N1"] = set("N1 C2 O2 N3 C4 O4 C5 C6 C7 OA OB".split())
usrNodeGroups["OMP"]["P"] = set("P OP1 OP2 OP3 O5' C5' C4' O4' C1' C3' C2' O2' O3'".split())

Now we load the information into the DNA object.

In [4]:
#################################
### Load info to object

dnap.setNumWinds(numWinds)
dnap.setNumSampledFrames(numSampledFrames)
dnap.setCutoffDist(cutoffDist)
dnap.setContactPersistence(contactPersistence)
dnap.setSolvNames(h2oName)
dnap.setSegIDs(segIDs)

dnap.setNodeGroups(usrNodeGroups)

# Load the trajectory

Now that all parameters are defined and loaded, we can start analyzing the simulation.

Our Generalized Network Analysis leverages the MDAnalysis package to create a *universe* that contains all the trajectory and system information.

In [5]:
dnap.loadSystem(psfFile,dcdFiles)

In [6]:
# We can access the trajectory data directly using the `getU` method.
dnap.getU().trajectory

<ChainReader containing decarboxylase.1.dcd with 1001 frames of 48341 atoms>

### Checks segments and residue names

It is important to know if there are residues in the structure that we are unaware of and need to be addressed. Otherwise, network analysis will not be able to create nodes in all selected residues.

In [7]:
dnap.checkSystem()

Residue verification:

---> SegID  ENZY : 19 unique residue types:
{'ARG', 'LYS', 'ALA', 'THR', 'PRO', 'GLU', 'VAL', 'GLY', 'TYR', 'SER', 'ASN', 'HSD', 'CYS', 'MET', 'ASP', 'PHE', 'ILE', 'LEU', 'GLN'}

---> SegID  OMP : 1 unique residue types:
{'OMP'}

---> 20 total selected residue types:
{'HSD', 'ARG', 'LYS', 'CYS', 'MET', 'OMP', 'ALA', 'ASP', 'THR', 'PHE', 'PRO', 'GLU', 'VAL', 'GLY', 'ILE', 'TYR', 'SER', 'ASN', 'LEU', 'GLN'}

---> 2 not-selected residue types in other segments:
{'TIP3', 'SOD'}

---> 22 total residue types:
{'TIP3', 'HSD', 'SOD', 'ARG', 'LYS', 'CYS', 'MET', 'OMP', 'ALA', 'ASP', 'THR', 'PHE', 'PRO', 'GLU', 'VAL', 'GLY', 'ILE', 'TYR', 'SER', 'ASN', 'LEU', 'GLN'}

---> 216 total residues were selected for network analysis.

Segments verification:

---> 2 selected segments:
['OMP', 'ENZY']

---> 9 not-selected segments:
['ION', 'WT1', 'WT2', 'WT3', 'WT4', 'WT5', 'WT6', 'WT7', 'WT8']



In [8]:
# More checks?
#
# You can add more checks here if your system needs special attention.

### Automatically identify crystallographic waters and ions.

In this section, we identify all residues which will be checked for connectivity with selected segments.

- First, we define if all solvent molecules will be checked or if just ions, ligands, lipids, and other molecules will be checked.

- Second, we sample a small set of frames from the trajectory to select residues likely to be structural. Then, we check the entire trajectory to see if the selected residues are closer than the cutoff distance for at least x% of the simulation (where x is the "contact persistence" fraction of the trajectory, which we defined above).

- Third and finally, we load the trajectory of the relevant atoms to memory to speed up the analysis.

In [9]:
dnap.selectSystem(withSolvent=True)

Checking 40 frames (striding 25)...


Frames:   0%|          | 0/40 [00:00<?, ?it/s]

14 extra residues will be added to the system.
The initial universe had 48341 atoms.
The final universe has 1673 atoms.
Loading universe to memory...


### Prepare network representation of the system

Here we check that we know how to treat all types of residues in the final selection. This is because every residue will generate one or more nodes in the final network. Then we store the groups of atoms that define each node.

In [10]:
dnap.prepareNetwork()

Nodes:   0%|          | 0/231 [00:00<?, ?it/s]

## Align the trajectory based on selected segments

We align the trajectory to its first frame using heavy atoms (non-hydrogen) from the selected segments. In the process, we also transfer the trajectory to the computer memory so that future analysis and manipulations are completed faster.

In [11]:
# If your system is too large, you can turn off the "in memory" option, at a cost for performance.
dnap.alignTraj(inMemory=True)

### Select residues that are closer than 4.5A for more than 75% of simulation

<img align="right" src="./toolkit/figures/nodeContactDistances.png" alt="drawing" width="400px"/>

The following cell calls efficient functions to analyze the system and create a contact matrix. We leverage both MDAnalysis parallel contact detection tools, as well as accelerated Numba and Cython functions. After creating the contact matrix, we remove any automatically selected nodes that have insufficient persistance and filter the contacts by (optionally) removing contacts between nodes in consecutive residues in a protein or nucleic chain.

In the example to the right, the distances between atoms indicate that in that frame of the simulation, the node representing the water molecule would make contact with the phosphate node (representing the ribose ring and phosphate) but not with the nitrogen node (representing the nucleic base).

Ultimately, an N-by-N matrix is created for all N nodes covering the selected region and automatically selected nodes (ions, solvent).


**Attention** For every time you start this Jupyter Notebook, the first time you execute this function may take significantly longer (several seconds) to start. This is because we use *Cython* and *Numba* to compile functions "on-demand", and a new compilation may be necessary after the notebook is restarted.

In [12]:
# To speed-up the contact matrix calculation, a larger stride can be selected, at a cost for precision.
dnap.findContacts(stride=1)

Window:   0%|          | 0/4 [00:00<?, ?it/s]

We found 0 nodes with no contacts.
We found 1136 contacting pairs out of 26565 total pairs of nodes.
(That's 4.3%, by the way)


### Removing contacts between nodes in the same residue.

The following function guarantees that there will be no "self-contacts" (contacts between a node and itself), and gives you the opportunity to remove contacts between nodes in consecutive residues (such as sequential amino acids in the same chain, removing back-bone interactions). 

The function also removes nodes that are never in contact with any other node in the system (such as the ends of flexible chains, or residues in flexible loops). This will automatically update the MDanalysis universe and related network information, such as number of nodes and atom-to-node mappings.

In [13]:
dnap.filterContacts(notSameRes=True, notConsecutiveRes=False, removeIsolatedNodes=True)

Filtering contacts in each window.


Window:   0%|          | 0/4 [00:00<?, ?it/s]

Node:   0%|          | 0/231 [00:00<?, ?it/s]

Node:   0%|          | 0/231 [00:00<?, ?it/s]

Node:   0%|          | 0/231 [00:00<?, ?it/s]

Node:   0%|          | 0/231 [00:00<?, ?it/s]


Removing isolated nodes...

We found 0 nodes with no contacts.
Running new contact matrix sanity check...
We found 0 nodes with no contacts.
We found 1135 contacting pairs out of 26565 total pairs of nodes.
(That's 4.3%, by the way)
Updating atom-to-node mapping...


Node:   0%|          | 0/231 [00:00<?, ?it/s]

## Calculate Generalized Correlation with Python/Numba

In [14]:
# We can calculate generalized correlations in parallel using Python's multiprocessing package.
dnap.calcCor(ncores=4)

Window:   0%|          | 0/4 [00:00<?, ?it/s]

Contact Pair:   0%|          | 0/1105 [00:00<?, ?it/s]

Contact Pair:   0%|          | 0/1121 [00:00<?, ?it/s]

Contact Pair:   0%|          | 0/1118 [00:00<?, ?it/s]

Contact Pair:   0%|          | 0/1108 [00:00<?, ?it/s]

## Calculate cartesian distances between all nodes in the selected system.

Here, we will calculate the **shortest** distance between atoms in all pairs of nodes. It is similar to the contact matrix calculation, but we check all distances and keep the shortest ones for our analysis.

In [15]:
# We can leverage MDanalysis parallelization options with backend="serial" or backend="openmp".
# For very small systems, the serial can be faster!
dnap.calcCartesian(backend="serial")

Calculating cartesian distances...

Sampling a total of 40 frames from 4 windows (10 per window)...


MEAN: Timesteps:   0%|          | 0/40 [00:00<?, ?it/s]

SEM/MIN/MAX: Timesteps:   0%|          | 0/40 [00:00<?, ?it/s]

# Network Calculations
Create a graph from our correlation matrix. Different properties are calculated:

*Density* measures how connected the graph is compared to how connected it *could* be. It is the ratio between edges in the graph over all possible edges between all pairs of nodes.

*Transitivity* measures the triadic closure, comparing present triangles to possible triangles. In a triangle, if A is connected to B, and B connected to C, then A is connected to C.

*Degree* measures the number of connections a node has.

(Reference)[1]

[1]:https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python#advanced-networkx-community-detection-with-modularity

In [16]:
dnap.calcGraphInfo()

In [17]:
# Basic information of the network as interpreted as a graph.
import networkx as nx
num_nodes = len(dnap.nxGraphs[0].nodes)
num_edges = len(dnap.nxGraphs[0].edges)

print(f"Graph with {num_nodes} nodes and {num_edges} edges")

Graph with 231 nodes and 905 edges


In [18]:
# Both density and transitivity are scaled from 0 to 1
for win in range(dnap.numWinds):
    print("----- Window {} -----".format(win))
    print("Density:", round( nx.density(dnap.nxGraphs[win]), 4) )
    print("Transitivity:", round( nx.transitivity(dnap.nxGraphs[win]), 4) )
    print()

----- Window 0 -----
Density: 0.0341
Transitivity: 0.4265

----- Window 1 -----
Density: 0.0344
Transitivity: 0.4266

----- Window 2 -----
Density: 0.0342
Transitivity: 0.4218

----- Window 3 -----
Density: 0.0335
Transitivity: 0.4227



In [19]:
from operator import itemgetter
from dynetan.toolkit import getSelFromNode

# We can check the nodes that have the most connections in each window.
for win in range(dnap.numWinds):
    print("----- Window {} -----".format(win))
    
    sorted_degree = sorted(dnap.getDegreeDict(win).items(), key=itemgetter(1), reverse=True)
    
    print("Top 10 nodes by degree: [node --> degree : selection]")
    for n,d in sorted_degree[:5]:
        print("{0:>4} --> {1:>2} : {2}".format(n, d, getSelFromNode(n, dnap.nodesAtmSel)))
    
    print()

----- Window 0 -----
Top 10 nodes by degree: [node --> degree : selection]
  48 --> 15 : resname PHE and resid 59 and segid ENZY
  36 --> 14 : resname LEU and resid 47 and segid ENZY
  69 --> 14 : resname ASN and resid 80 and segid ENZY
  77 --> 14 : resname PHE and resid 88 and segid ENZY
 111 --> 13 : resname LEU and resid 122 and segid ENZY

----- Window 1 -----
Top 10 nodes by degree: [node --> degree : selection]
  36 --> 14 : resname LEU and resid 47 and segid ENZY
  48 --> 14 : resname PHE and resid 59 and segid ENZY
  77 --> 14 : resname PHE and resid 88 and segid ENZY
  95 --> 14 : resname VAL and resid 106 and segid ENZY
  37 --> 13 : resname VAL and resid 48 and segid ENZY

----- Window 2 -----
Top 10 nodes by degree: [node --> degree : selection]
  48 --> 16 : resname PHE and resid 59 and segid ENZY
  36 --> 13 : resname LEU and resid 47 and segid ENZY
  37 --> 13 : resname VAL and resid 48 and segid ENZY
  77 --> 13 : resname PHE and resid 88 and segid ENZY
 157 --> 13 : r

## Calculate optimal paths
We choose the Floyd Warshall algorithm[1]. Optimal path calculation uses the **correlations as weights** to determine network distances and shortest distances.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.dense.floyd_warshall.html?highlight=warshall#networkx.algorithms.shortest_paths.dense.floyd_warshall

In [20]:
dnap.calcOptPaths(ncores=4)

Window:   0%|          | 0/4 [00:00<?, ?it/s]

## Calculate betweenness

We calculate both betweenness centrality[1] for edges and eigenvector centrality[2] for nodes.

[1]:https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html?highlight=betweenness#networkx.algorithms.centrality.edge_betweenness_centrality
[2]:https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.centrality.eigenvector_centrality.html

In [21]:
dnap.calcBetween(ncores=4)

Window:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
from itertools import islice

# Pairs of nodes with highest Betweeness values compared to their correlation values in Window 0.
for k,v in islice(dnap.btws[0].items(),5):
    msg_str = "Nodes {} have betweenes {} and correlation {}."
    print(msg_str.format(k, round(v,3), round(dnap.corrMatAll[0, k[0], k[1]], 3) ) )

Nodes (61, 115) have betweenes 0.041 and correlation 0.072.
Nodes (31, 216) have betweenes 0.034 and correlation 0.137.
Nodes (30, 31) have betweenes 0.031 and correlation 0.656.
Nodes (59, 215) have betweenes 0.031 and correlation 0.041.
Nodes (31, 85) have betweenes 0.03 and correlation 0.07.


Turn to node centrality instead of edge centrality:

In [23]:
dnap.calcEigenCentral()

### Calculate communities



Using **Louvain heuristices** is feasible. 
This method also maximizes the modularity of the network.

http://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008/meta

In [24]:
dnap.calcCommunities()

In [25]:
# Sort communities based on number of nodes
for comIndx in dnap.nodesComm[0]["commOrderSize"]:
    print("Modularity Class {0:>2}: {1:>3} nodes.".format(comIndx, len(dnap.nodesComm[0]["commNodes"][comIndx])))

Modularity Class  3:  30 nodes.
Modularity Class  2:  27 nodes.
Modularity Class  4:  23 nodes.
Modularity Class  5:  22 nodes.
Modularity Class  1:  20 nodes.
Modularity Class 11:  20 nodes.
Modularity Class  9:  19 nodes.
Modularity Class  7:  18 nodes.
Modularity Class 10:  16 nodes.
Modularity Class  0:  15 nodes.
Modularity Class 12:   9 nodes.
Modularity Class 13:   9 nodes.
Modularity Class  6:   1 nodes.
Modularity Class  8:   1 nodes.
Modularity Class 14:   1 nodes.


In [26]:
# Sort communities based on the node with highest eigenvector centrality
for comIndx in dnap.nodesComm[0]["commOrderEigenCentr"]:
    print("Modularity Class {0} ({1} nodes) Sorted by Eigenvector Centrality:".format(
                                                                    comIndx, 
                                                                len(dnap.nodesComm[0]["commNodes"][comIndx])))
    for node in dnap.nodesComm[0]["commNodes"][comIndx][:5]:
        print("Name: {0:>4} | Degree: {1:>2} | Eigenvector Centrality: {2}".format(
            node, dnap.nxGraphs[win].nodes[node]['degree'], dnap.nxGraphs[win].nodes[node]['eigenvector']))
    print()

Modularity Class 9 (19 nodes) Sorted by Eigenvector Centrality:
Name:  149 | Degree: 10 | Eigenvector Centrality: 0.28339865700890216
Name:  150 | Degree:  7 | Eigenvector Centrality: 0.2657041915897851
Name:  147 | Degree: 11 | Eigenvector Centrality: 0.30480602404681556
Name:  146 | Degree: 10 | Eigenvector Centrality: 0.2677439346113377
Name:  148 | Degree:  7 | Eigenvector Centrality: 0.28682742744913714

Modularity Class 10 (16 nodes) Sorted by Eigenvector Centrality:
Name:  153 | Degree: 14 | Eigenvector Centrality: 0.16689946188013058
Name:  156 | Degree: 12 | Eigenvector Centrality: 0.2072584478392879
Name:  152 | Degree:  9 | Eigenvector Centrality: 0.16490506113332604
Name:  155 | Degree:  9 | Eigenvector Centrality: 0.18836706805184342
Name:  154 | Degree:  7 | Eigenvector Centrality: 0.16325548793315683

Modularity Class 7 (18 nodes) Sorted by Eigenvector Centrality:
Name:  115 | Degree:  9 | Eigenvector Centrality: 0.05499665207776102
Name:  118 | Degree:  7 | Eigenvector 

# Process Interface Residues

We now find all nodes that are close to both selections chosen by the user. That may include amino acids in the interface, as well as ligands, waters, and ions.

In [27]:
# This method returns the number of unique nodes in interface node pairs.
num_nodes = dnap.interfaceAnalysis(selAstr="segid ENZY", selBstr="segid OMP")
print(f"There are {num_nodes} nodes making contacts along the complex interface.")

Samples:   0%|          | 0/10 [00:00<?, ?it/s]

There are 55 nodes making contacts along the complex interface.


## Save The Data

In [28]:
pathToData = "./TutorialResults/"

fileNameRoot = "dnaData"

fullPathRoot = os.path.join(pathToData, fileNameRoot)

dnap.saveData(fullPathRoot)

## Save reduced trajectory for visualization

In [29]:
# Now we will save a reduced DCD trajectory with the heavy atoms used for network analysis
# A smaller trajectory can be created by choosing a "stride" that sub-samples the original trajectory.
# This function will also produce a PDB file so that information on atoms and residues can be loaded to
#    visualization software such as VMD.

dcd_stride = 1
num_atoms = dnap.workU.atoms.n_atoms
num_frames = len(dnap.workU.trajectory[::dcd_stride])

print(f"We will save {num_atoms} heavy atoms and {num_frames} frames.")

We will save 1673 heavy atoms and 1001 frames.


MDAnalysis may print warnings regarding missing data fields, such as altLocs, icodes, occupancies, or tempfactor, which provide information commonly found in PDB files.
The warnings are for your information, and in the context of this tutorial, they are expected and do not indicate a problem. We will silence such "UserWarning"s for clarity.

In [30]:
# Suppress some MDAnalysis warnings about writing PDB files
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Save PDB (for topology) and DCD (for coordinates) files with reduced trajectory.
dnap.saveReducedTraj(fullPathRoot, stride = dcd_stride)

Frames:   0%|          | 0/1001 [00:00<?, ?it/s]

# Analysis

We have finished processing the trajectory and storing all related data. We can now move on to analysis
of the network properties calculated here.

**All analysis code was placed in a second tutorial notebook for clarity.**


# ---- The End ----